<img src="https://datasciencedegree.wisconsin.edu/wp-content/themes/data-gulp/images/logo.svg" width="300">


This file is Stage - 1 of project which involves 
- use tweepy API
- collect 5000 tweets
- extract text from tweets
- create dataframe from tweet text
- create two sorted and unique lists as pre_obesity and post_obesity
- Remove duplicate tweets from dataframe
- Store dataframe as csv file

Study is done to test that number of words related to preventive aspect of obesity will be same as that of disease aspect of obesity.


In [1]:
# function used to find number of substring from list in string
def counter(s, lst):
    s = s.lower()
    return sum(s.count(sub) for sub in lst)

# create two lists for obesity
pre_obesity = ['food',
              'breakfast',
               'lunch',
               'dinner',
               'protein',
               'apetite',
               'potassium',
               'vitamin',
               'energy',
               'starving',
               'berry',
               'coffee',
               'alcohol',
               'caffeine',
               'sugar',
               'milk',
               'cigarette',
               'diet',
               'sleep',
               'calorie',
               'run',
               'walk',
               'nutrition',
               'tea',
               'snack',
               'meal',
               'fitness',
               'keto'
               
              ]

# removing any duplicates if present
pre_obesity = list(set(pre_obesity))
pre_obesity = sorted(pre_obesity)

# This is second list to compare
post_obesity = ['diabetes',
                'heart',
                'brain',
               'kidney',
                'surgery',
                'arthritis',
                'cancer',
                'disease',
                'inflammation',
                'endocrine',
                'gastric',
                'liver',
                'pregnancy',
                'metabolic',
                'syndrome',
                'fertility',
                'pancreas',
                'pain',
                'ache',
                'oral'
               ]

post_obesity = list(set(post_obesity))
post_obesity = sorted(post_obesity)

In [2]:
# import the packages
import tweepy
import pandas as pd

In [3]:
# run the text file with credentials
%run ~/Desktop/twitter_credentials.py

In [4]:
#Use tweepy.OAuthHandler to create an authentication using the given key and secret
auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
auth.set_access_token(acc_token, acc_secret)

#Connect to the Twitter API using the authentication
api = tweepy.API(auth)

In [5]:
# collecting more tweets
num_needed = 5000
tweet_list = []
last_id = -1 # id of last tweet seen
while len(tweet_list) < num_needed:
    try:
        new_tweets = api.search(q = '#%23obesity', tweet_mode='extended',
                                count = 100, max_id = str(last_id - 1))
    except tweepy.TweepError as e:
        print("Error", e)
        break
    else:
        if not new_tweets:
            print("Could not find any more tweets!")
            break
        tweet_list.extend(new_tweets)
        last_id = new_tweets[-1].id
        
tweet_text_list = [tweet.full_text for tweet in tweet_list]
len(tweet_list)

Could not find any more tweets!


4044

In [6]:
# create dataframe from tweet text
df_current = pd.DataFrame(tweet_text_list, columns= ['Tweet_text'])
df_current.count()
df_current.head()

,Tweet_text
0,Another publication from the @abetterstartnz i...
1,RT @yourcroydon: Great cross borough turnout a...
2,What's leading to shorter lifespans in America...
3,VIDEO: Clinical studies of obesity in oncogene...
4,RT @OrganicLiveFood: Various studies have show...


In [7]:
#execute it only once per session
df = pd.DataFrame(columns= ['Tweet_text'])

In [8]:
# creating DataFrame to hold the results

df = pd.concat([df_current, df])
df.count()

Tweet_text    4044
dtype: int64

In [9]:
# remove duplicate values of tweet text
df = df.drop_duplicates()
df.count()

Tweet_text    2447
dtype: int64

In [15]:
# create column of true / false depending on text contains #obesity

df['obesity_present'] = df.Tweet_text.apply(lambda row: 'obesity' in row.lower() ) 
df.head(5)

,Tweet_text,obesity_present
0,Another publication from the @abetterstartnz i...,True
1,RT @yourcroydon: Great cross borough turnout a...,False
2,What's leading to shorter lifespans in America...,True
3,VIDEO: Clinical studies of obesity in oncogene...,True
4,RT @OrganicLiveFood: Various studies have show...,False


In [16]:
# filter dataframe where obesity_present is true
df = df[df['obesity_present'] == True]
df.count()

Tweet_text         2163
obesity_present    2163
dtype: int64

In [17]:
# create column of count of pre-obesity
df['pre_count'] = df['Tweet_text'].apply( lambda row : counter(row, pre_obesity) )
df.head()

C:\Users\aniruddha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Tweet_text,obesity_present,pre_count
0,Another publication from the @abetterstartnz i...,True,0
2,What's leading to shorter lifespans in America...,True,0
3,VIDEO: Clinical studies of obesity in oncogene...,True,0
5,"RT @SlimmingWorld: Ahead of #GE2019, we're cal...",True,0
6,RT @rcpsychEDFac: There needs to be better dia...,True,0


In [18]:
# create column of count of post-obesity
df['post_count'] = df['Tweet_text'].apply( lambda row : counter(row, post_obesity) )
df.head()

C:\Users\aniruddha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Tweet_text,obesity_present,pre_count,post_count
0,Another publication from the @abetterstartnz i...,True,0,0
2,What's leading to shorter lifespans in America...,True,0,1
3,VIDEO: Clinical studies of obesity in oncogene...,True,0,1
5,"RT @SlimmingWorld: Ahead of #GE2019, we're cal...",True,0,0
6,RT @rcpsychEDFac: There needs to be better dia...,True,0,0


In [19]:
sum(df['pre_count'])

1473

In [20]:
sum(df['post_count'])

1526

In [21]:
df.to_csv(r'obesity.csv')